In [14]:
import numpy as np
import pandas as pd
%matplotlib inline 
# Option to display all dataframes columns
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

housing = pd.read_csv('Ames_HousePrice_cleaned.csv', index_col=0)
housing.shape

(2580, 59)

In [22]:
#Whole data set, no split
from sklearn.linear_model import LinearRegression

X=housing.drop(columns='SalePrice')
y=housing.SalePrice
lin = LinearRegression().fit(X,y)
print(lin.score(X,y))

0.8876308381267729


In [2]:
from sklearn.model_selection import train_test_split

X=housing.drop(columns='SalePrice')
y=housing.SalePrice
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=42,shuffle=True)

In [3]:
#Baseline model with all variables
from sklearn.linear_model import LinearRegression

lin = LinearRegression().fit(X_train,y_train)
print(lin.score(X_train,y_train))
print(lin.score(X_test,y_test))

0.8864374204793761
0.8864374998432968


In [4]:
#Baseline model with all variables (90-10 train test split)
from sklearn.linear_model import LinearRegression

lin = LinearRegression().fit(X_train,y_train)
print(lin.score(X_train,y_train))
print(lin.score(X_test,y_test))

0.8864374204793761
0.8864374998432968


In [5]:
#K fold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

X=housing.drop(columns='SalePrice')
y=housing.SalePrice
kf = KFold(n_splits=5,shuffle=True,random_state=42)

for train_idx, test_idx in kf.split(X,y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    lin = LinearRegression().fit(X_train,y_train)
    print(lin.score(X_train,y_train))
    print(lin.score(X_test,y_test))



0.8864374204793761
0.8864374998432968
0.8980440677676648
0.8405802616258292
0.8861699444580762
0.8886632274573878
0.8848221966009314
0.894772876324662
0.8880798987466282
0.8792946363016549


In [73]:
from sklearn.model_selection import KFold
import statsmodels.api as sm 


X= housing.drop(columns='SalePrice')
X= sm.add_constant(X)
y= housing.SalePrice
kf = KFold(n_splits=5,shuffle=True,random_state=42)
results=pd.DataFrame(index=housing.columns)

for train_idx, test_idx in kf.split(X,y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    ols = sm.OLS(y_train, X_train)
    ans = ols.fit()
    print(ans.rsquared)
    results= pd.concat([results,ans.pvalues],axis=1)
results['Mean']=results.mean(axis=1)
results.sort_values(['Mean'],ascending=False)


0.8864374204793761
0.8980440677676649
0.8861699444580762
0.8848221966009314
0.8880798987466283


,0,0,0,0,0,Mean
FireplaceQu,8.096507e-01,9.489494e-01,9.052294e-01,4.383610e-01,9.765548e-01,8.157491e-01
YearRemodAdd,6.485412e-01,9.755034e-01,9.716150e-01,3.142082e-01,9.467709e-01,7.713277e-01
3SsnPorch,7.761081e-01,6.793531e-01,9.410952e-01,4.871255e-01,9.395985e-01,7.646561e-01
PavedDrive,7.755261e-01,9.563179e-01,7.082893e-01,6.582061e-01,6.806121e-01,7.557903e-01
Class_1.5story,9.927920e-01,6.703099e-01,5.924804e-01,9.778888e-01,5.005962e-01,7.468135e-01
Baths,5.731020e-01,7.684513e-01,8.458439e-01,9.843909e-01,5.238145e-01,7.391205e-01
ExterCond,2.136511e-01,9.547990e-01,8.818727e-01,4.712412e-01,8.853337e-01,6.813795e-01
GarageQual,6.208846e-01,3.203128e-01,5.227863e-01,8.876588e-01,7.311600e-01,6.165605e-01
Fence,8.125390e-01,6.644502e-01,5.057891e-01,4.619880e-01,4.421461e-01,5.773825e-01
GarageCars,5.345200e-01,8.913593e-01,2.227036e-01,5.983303e-01,5.301194e-01,5.554065e-01


In [74]:
from sklearn.model_selection import KFold
import statsmodels.api as sm 
from sklearn.preprocessing import StandardScaler

#Get features and response
X= housing.drop(columns='SalePrice')
y= housing.SalePrice.to_numpy().reshape(-1,1)

#Scale X and y
scaler_X = StandardScaler().fit(X)
X_scaled = scaler_X.transform(X)
scaler_y = StandardScaler().fit(y)
y_scaled = scaler_y.transform(y)

#Convert back to dataframes
X= pd.DataFrame(data=X_scaled,index=X.index,columns=X.columns)
y= pd.DataFrame(data=y_scaled,index=X.index,columns=['SalePrice'])

#Add constant for stats model API
X= sm.add_constant(X)

#Initialize empty results dataframe and K-folds object
kf = KFold(n_splits=5,shuffle=True,random_state=42)
results=pd.DataFrame(index=housing.columns)

#Train statsmodel objects
for train_idx, test_idx in kf.split(X,y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    ols = sm.OLS(y_train, X_train)
    ans = ols.fit()
    print(ans.rsquared)
    results= pd.concat([results,ans.pvalues],axis=1)
    #print(ans.pvalues.sort_values(ascending=False))
results['Mean']=results.mean(axis=1)
results.sort_values(['Mean'],ascending=False)


0.8864374204793761
0.8980440677676649
0.8861699444580762
0.8848221966009313
0.8880798987466282


,0,0,0,0,0,Mean
FireplaceQu,8.096507e-01,9.489494e-01,9.052294e-01,4.383610e-01,9.765548e-01,8.157491e-01
YearRemodAdd,6.485412e-01,9.755034e-01,9.716150e-01,3.142082e-01,9.467709e-01,7.713277e-01
3SsnPorch,7.761081e-01,6.793531e-01,9.410952e-01,4.871255e-01,9.395985e-01,7.646561e-01
PavedDrive,7.755261e-01,9.563179e-01,7.082893e-01,6.582061e-01,6.806121e-01,7.557903e-01
Class_1.5story,9.927920e-01,6.703099e-01,5.924804e-01,9.778888e-01,5.005962e-01,7.468135e-01
const,7.689250e-01,7.146174e-01,5.902295e-01,8.581581e-01,7.780947e-01,7.420049e-01
Baths,5.731020e-01,7.684513e-01,8.458439e-01,9.843909e-01,5.238145e-01,7.391205e-01
ExterCond,2.136511e-01,9.547990e-01,8.818727e-01,4.712412e-01,8.853337e-01,6.813795e-01
GarageQual,6.208846e-01,3.203128e-01,5.227863e-01,8.876588e-01,7.311600e-01,6.165605e-01
Fence,8.125390e-01,6.644502e-01,5.057891e-01,4.619880e-01,4.421461e-01,5.773825e-01


In [76]:
#Simple stats model API on entire dataset
import statsmodels.api as sm 

X=housing.drop(columns='SalePrice')
y=housing.SalePrice

X_add_const = sm.add_constant(X)
ols = sm.OLS(y, X_add_const)
ans = ols.fit()
print(ans.summary())
print(ans.rsquared)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     349.5
Date:                Sun, 22 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:43:08   Log-Likelihood:                -29803.
No. Observations:                2580   AIC:                         5.972e+04
Df Residuals:                    2522   BIC:                         6.006e+04
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -7.121e+05   9

In [ ]:
#First round of drops
